In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import spacy
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from alibi.explainers import AnchorTabular, AnchorText
from alibi.datasets import fetch_adult, fetch_movie_sentiment
from alibi.utils import gen_category_map, DistilbertBaseUncased, BertBaseUncased, RobertaBase

c:\Users\admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\admin\anaconda3\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Users\admin\anaconda3\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [2]:
adult = fetch_adult()
adult.keys()

dict_keys(['data', 'target', 'feature_names', 'target_names', 'category_map'])

In [3]:
adult['category_map']

{1: ['?',
  'Federal-gov',
  'Local-gov',
  'Never-worked',
  'Private',
  'Self-emp-inc',
  'Self-emp-not-inc',
  'State-gov',
  'Without-pay'],
 2: ['Associates',
  'Bachelors',
  'Doctorate',
  'Dropout',
  'High School grad',
  'Masters',
  'Prof-School'],
 3: ['Married', 'Never-Married', 'Separated', 'Widowed'],
 4: ['?',
  'Admin',
  'Blue-Collar',
  'Military',
  'Other',
  'Professional',
  'Sales',
  'Service',
  'White-Collar'],
 5: ['Husband',
  'Not-in-family',
  'Other-relative',
  'Own-child',
  'Unmarried',
  'Wife'],
 6: ['Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'],
 7: ['Female', 'Male'],
 11: ['?',
  'British-Commonwealth',
  'China',
  'Euro_1',
  'Euro_2',
  'Latin-America',
  'Other',
  'SE-Asia',
  'South-America',
  'United-States',
  'Yugoslavia']}

In [4]:
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

In [5]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:, :-1]
target = data_perm[:, -1]
idx = 30000
X_train, y_train = data[:idx, :], target[:idx]
X_test, y_test = data[idx:, :], target[idx:]

In [6]:
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer', IterativeImputer()), ('scaler', StandardScaler())])

In [8]:
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [9]:
preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer, ordinal_features), ('cat', categorical_transformer, categorical_features)])
preprocessor.fit(X_train)

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,estimator,None
,missing_values,nan
,sample_posterior,False


In [20]:
np.random.seed(0)
clf = RandomForestClassifier(max_depth=20, n_jobs=-1)
clf.fit(preprocessor.transform(X_train), y_train)
clf.score(preprocessor.transform(X_train), y_train), clf.score(preprocessor.transform(X_test), y_test)

(0.9206666666666666, 0.8691917219836002)

In [21]:
predict_fn = lambda x: clf.predict(preprocessor.transform(x))

In [22]:
explainer = AnchorTabular(predict_fn, feature_names=feature_names, categorical_names=category_map, seed=1)
explainer.fit(X_train, disc_perc=[25, 50, 75])

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': 1, 'disc_perc': [25, 50, 75]},
  'version': '0.9.6'}
)

In [33]:
idx = 0
class_names = adult.target_names
cls = explainer.predictor(X_test[idx].reshape(1, -1))
class_names[cls[0]]

'<=50K'

In [25]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
explanation.anchor, explanation.precision, explanation.coverage

(['Marital Status = Separated', 'Education = High School grad'],
 0.9772727272727273,
 0.1109)

In [37]:
idx = 7
cls = explainer.predictor(X_test[idx].reshape(1, -1))
explanation = explainer.explain(X_test[idx], threshold=0.95)
class_names[cls[0]], explanation.anchor, explanation.precision, explanation.coverage

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


('>50K',
 ['Capital Loss > 0.00',
  'Marital Status = Married',
  'Relationship = Husband',
  'Age > 47.00',
  'Race = White',
  'Country = United-States'],
 0.7155887230514096,
 0.0089)

In [38]:
movies = fetch_movie_sentiment()
movies.keys()

dict_keys(['data', 'target', 'target_names'])

In [39]:
data = movies.data
labels = movies.target
target_names = movies.target_names

In [40]:
train, test, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)
train, val, train_labels, val_labels = train_test_split(train, train_labels, test_size=0.1, random_state=42)
train_labels, test_labels, val_labels = np.array(train_labels), np.array(test_labels), np.array(val_labels)

In [41]:
vect = CountVectorizer()
vect.fit(train)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [54]:
np.random.seed(0)
clf = LogisticRegression(solver='liblinear', C=1)
clf.fit(vect.transform(train), train_labels)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


In [55]:
predict_fn = lambda x: clf.predict(vect.transform(x))

In [56]:
preds_train, preds_val, preds_test = predict_fn(train), predict_fn(val), predict_fn(test)
accuracy_score(preds_test, test_labels)

0.7589841878294202

In [59]:
nlp = spacy.load('en_core_web_md')

In [64]:
text = data[4]
pred = target_names[predict_fn([text])[0]]
alternative = target_names[1 - predict_fn([text])[0]]
pred

'negative'

In [65]:
explainer = AnchorText(predict_fn, sampling_strategy='unknown', nlp=nlp)
explainer

AnchorText(meta={
  'name': 'AnchorText',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': 0, 'sample_proba': 0.5},
  'version': '0.9.6'}
)

In [66]:
explanation = explainer.explain(text, threshold=0.95)
explanation.anchor, explanation.precision, explanation.coverage

(['flashy'], 0.99375, 0.4993)

In [67]:
[x for x in explanation.raw['examples'][-1]['covered_true']]

['a UNK flashy UNK UNK opaque and emotionally vapid exercise in style UNK mystification .',
 'a UNK flashy UNK UNK UNK and emotionally UNK exercise UNK UNK and UNK UNK',
 'a UNK flashy UNK narratively opaque UNK UNK UNK exercise in style and UNK UNK',
 'UNK visually flashy UNK narratively UNK and emotionally UNK UNK UNK UNK UNK mystification .',
 'UNK UNK flashy UNK UNK opaque and emotionally UNK UNK in UNK and UNK .',
 'a visually flashy but UNK UNK and UNK UNK UNK in style UNK mystification .',
 'a visually flashy but UNK opaque UNK emotionally vapid UNK in UNK and mystification .',
 'a UNK flashy but narratively UNK UNK emotionally vapid exercise in style UNK mystification UNK',
 'a UNK flashy but narratively opaque UNK emotionally vapid exercise in style and mystification .',
 'a visually flashy UNK UNK opaque UNK UNK UNK exercise in UNK UNK UNK .']

In [68]:
[x for x in explanation.raw['examples'][-1]['covered_false']]

['UNK UNK flashy but narratively UNK and UNK UNK UNK in style and UNK UNK']

In [69]:
explainer = AnchorText(predict_fn, sampling_strategy='similarity', sample_proba=0.5, nlp=nlp)
explainer

AnchorText(meta={
  'name': 'AnchorText',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {
              'seed': 0,
              'sample_proba': 0.5,
              'top_n': 100,
              'temperature': 1.0,
              'use_proba': False}
            ,
  'version': '0.9.6'}
)

In [70]:
explanation = explainer.explain(text, threshold=0.95)
explanation.anchor, explanation.precision, explanation.coverage

(['exercise', 'vapid'], 0.9883040935672515, 0.2513)

In [71]:
[x for x in explanation.raw['examples'][-1]['covered_false']]

['a visually punchy but tragically opaque and hysterically vapid exercise in minimalist and mystification .',
 'a visually discernible but realistically posh and physically vapid exercise around style and determination .']